# Model Development - House Price Dataset
## Introduction
This notebook develops and compares multiple machine learning models for house price prediction. Using the engineered features from `03_feature_engineering.ipynb`, I will train, evaluate, and select the best performing model.

**Dataset:** Housing Price Prediction Data (Kaggle)

**Objective:** Train, tune, and compare regression models to predict house prices as accurately as possible.

**Author:** NGUYEN Ngoc Dang Nguyen - Final-year Student in Computer Science, Aix-Marseille University

**Model development steps:**
1. Load engineered data and prepare for modeling
2. Split data into training and testing sets
3. Train multiple regression algorithms
4. Evaluate model performance using multiple metrics
5. Compare models and select the best one
6. Analyze feature importance and model for deployment
7. Save the final model for deployment

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Load the engineered dataset
df = pd.read_csv("../data/processed/modeling_data.csv")

print(f"Modeling dataset loaded: {df.shape[0]} rows and {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.1f} KB")

# Load feature engineering summary to get target column and display
import json
with open('../data/processed/feature_engineering_summary.json', 'r') as f:
    feature_info = json.load(f)

target_col = feature_info['target_column']
print(f"Target variable: {target_col}")

print(f"\nSample of the cleaned data:")
df.head()

## 2. Data Preparation for Modeling

In [ ]:
print("PREPARING DATA FOR MODELING")
print("=" * 40)

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

print(f"Dataset summary:")
print(f"    * Features (X): {X.shape[1]} columns")
print(f"    * Target (y): {y.shape[0]} values")
print(f"    * Target range: {y.min():.2f} to {y.max():.2f}")
print(f"    * Target mean: {y.mean():.2f}")

# Check for any remaining issues
print(f"\nData quality check:")
print(f"    * Missing values in X: {X.isnull().sum().sum()}")
print(f"    * Missing values in y: {y.isnull().sum()}")
print(f"    * Infinite values in X: {np.isinf(X.select_dtypes(include=[np.number])).sum().sum()}")

# Handle any infinite values
X = X.replace([np.inf, -np.inf], np.nan)
if X.isnull().sum().sum() > 0:
    print(f"Found {X.isnull().sum().sum()} missing/infinite values - filling with median")
    X = X.fillna(X.median())

# Feature types analysis
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\nFeature types:")
print(f"    * Numeric features: {len(numeric_features)}")
print(f"    * Categorical features: {len(categorical_features)}")

# Handle categorical features if any
if categorical_features:
    print(f"  Encoding categorical features...")
    X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)
    print(f"  After encoding: {X_encoded.shape[1]} features")
    X = X_encoded

## 3. Train-Test Split

In [ ]:
print("\nSPLITTING DATA FOR TRAINING AND TESTING")
print("=" * 40)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print(f"Data split summary:")
print(f"  * Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  * Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"  * Features: {X_train.shape[1]}")

# Check target distribution in splits
print(f"\nTarget distribution:")
print(f"  * Training mean: {y_train.mean():.2f}")
print(f"  * Test mean: {y_test.mean():.2f}")
print(f"  * Training std: {y_train.std():.2f}")
print(f"  * Test std: {y_test.std():.2f}")

# Visualize the split
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.hist(y_train, bins=30, alpha=0.7, label='Train', color='blue')
plt.hist(y_test, bins=30, alpha=0.7, label='Test', color='orange')
plt.title('Target Distribution: Train vs Test')
plt.xlabel(target_col)
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 3, 2)
plt.scatter(range(len(y_train)), sorted(y_train), alpha=0.6, s=1, label='Train')
plt.scatter(range(len(y_test)), sorted(y_test), alpha=0.6, s=1, label='Test')
plt.title('Target Values Distribution')
plt.xlabel('Sample Index')
plt.ylabel(target_col)
plt.legend()

plt.subplot(1, 3, 3)
plt.boxplot([y_train, y_test], labels=['Train', 'Test'])
plt.title('Target Distribution Comparison')
plt.ylabel(target_col)

plt.tight_layout()
plt.show()

## 4. Model Selection and Training

In [ ]:
print("\nMODEL TRAINING AND COMPARISON")
print("=" * 50)

# Define models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Store results
results = {}
trained_models = {}

print("Training models...")
print("-" * 30)

# Train and evaluate each model
for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train the model
    model.fit(X_train, y_train)
    trained_models[name] = model
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())
    
    # Store results
    results[name] = {
        'Train_MAE': train_mae,
        'Test_MAE': test_mae,
        'Train_RMSE': train_rmse,
        'Test_RMSE': test_rmse,
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'CV_RMSE': cv_rmse
    }
    
    print(f"  {name} completed:")
    print(f"     * Test R²: {test_r2:.3f}")
    print(f"     * Test RMSE: {test_rmse:.2f}")
    print(f"     * Test MAE: {test_mae:.2f}")

print(f"\nAll {len(models)} models trained successfully!")

## 5. Model Performance Comparison

In [ ]:
print("\nMODEL PERFORMANCE COMPARISON")
print("=" * 50)

# Create results DataFrame
results_df = pd.DataFrame(results).T
results_df = results_df.round(3)

print("Complete Results Table:")
print(results_df.to_string())

# Find best model
best_model_name = results_df['Test_R2'].idxmax()
best_model = trained_models[best_model_name]
best_r2 = results_df.loc[best_model_name, 'Test_R2']

print(f"\nBest Model: {best_model_name}")
print(f"   * Test R²: {best_r2:.3f}")
print(f"   * Test RMSE: {results_df.loc[best_model_name, 'Test_RMSE']:.2f}")
print(f"   * Test MAE: {results_df.loc[best_model_name, 'Test_MAE']:.2f}")

# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# R² Score comparison
axes[0, 0].bar(results_df.index, results_df['Test_R2'], color='skyblue', alpha=0.8)
axes[0, 0].set_title('Model Comparison - R² Score')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# RMSE comparison
axes[0, 1].bar(results_df.index, results_df['Test_RMSE'], color='lightcoral', alpha=0.8)
axes[0, 1].set_title('Model Comparison - RMSE')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Training vs Test R²
train_r2 = results_df['Train_R2']
test_r2 = results_df['Test_R2']
x_pos = np.arange(len(results_df))

axes[1, 0].bar(x_pos - 0.2, train_r2, 0.4, label='Train R²', alpha=0.8)
axes[1, 0].bar(x_pos + 0.2, test_r2, 0.4, label='Test R²', alpha=0.8)
axes[1, 0].set_title('Train vs Test R² Score')
axes[1, 0].set_ylabel('R² Score')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels(results_df.index, rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Cross-validation RMSE
axes[1, 1].bar(results_df.index, results_df['CV_RMSE'], color='lightgreen', alpha=0.8)
axes[1, 1].set_title('Cross-Validation RMSE')
axes[1, 1].set_ylabel('CV RMSE')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Best Model Analysis and Feature Importance

In [ ]:
print(f"\nANALYZING BEST MODEL: {best_model_name}")
print("=" * 50)

# Make predictions with best model
y_pred = best_model.predict(X_test)

# Detailed performance analysis
print(f"Detailed Performance Metrics:")
print(f"  * Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"  * Root Mean Square Error: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")
print(f"  * R² Score: {r2_score(y_test, y_pred):.3f}")
print(f"  * Mean Absolute Percentage Error: {np.mean(np.abs((y_test - y_pred) / y_test)) * 100:.2f}%")

# Feature importance analysis
if hasattr(best_model, 'feature_importances_'):
    print(f"\nFeature Importance Analysis:")
    
    # Get feature importances
    importances = best_model.feature_importances_
    feature_names = X.columns
    
    # Create feature importance DataFrame
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Most Important Features:")
    for i, (_, row) in enumerate(feature_importance_df.head(10).iterrows(), 1):
        print(f"{i:2d}. {row['feature']:<30} | {row['importance']:.3f}")
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    top_features = feature_importance_df.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top 15 Feature Importances - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

elif hasattr(best_model, 'coef_'):
    print(f"\nLinear Model Coefficients Analysis:")
    
    # Get coefficients for linear models
    coef = best_model.coef_
    feature_names = X.columns
    
    # Create coefficients DataFrame
    coef_df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coef,
        'abs_coefficient': np.abs(coef)
    }).sort_values('abs_coefficient', ascending=False)
    
    print(f"\nTop 10 Features by Coefficient Magnitude:")
    for i, (_, row) in enumerate(coef_df.head(10).iterrows(), 1):
        print(f"{i:2d}. {row['feature']:<30} | {row['coefficient']:>8.3f}")
    
    # Plot coefficients
    plt.figure(figsize=(12, 8))
    top_features = coef_df.head(15)
    colors = ['red' if x < 0 else 'blue' for x in top_features['coefficient']]
    plt.barh(range(len(top_features)), top_features['coefficient'], color=colors, alpha=0.7)
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Coefficient Value')
    plt.title(f'Top 15 Feature Coefficients - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 7. Model Validation and Error Analysis

In [ ]:
print(f"\nMODEL VALIDATION AND ERROR ANALYSIS")
print("=" * 50)

# Prediction vs Actual plots
plt.figure(figsize=(15, 5))

# Scatter plot: Predicted vs Actual
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel(f'Actual {target_col}')
plt.ylabel(f'Predicted {target_col}')
plt.title('Predicted vs Actual Values')
plt.grid(True, alpha=0.3)

# Residual plot
residuals = y_test - y_pred
plt.subplot(1, 3, 2)
plt.scatter(y_pred, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel(f'Predicted {target_col}')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)

# Residual distribution
plt.subplot(1, 3, 3)
plt.hist(residuals, bins=30, alpha=0.7, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.axvline(x=0, color='r', linestyle='--')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Error statistics
print(f"Error Analysis:")
print(f"  * Mean Residual: {residuals.mean():.3f}")
print(f"  * Std of Residuals: {residuals.std():.3f}")
print(f"  * Min Residual: {residuals.min():.2f}")
print(f"  * Max Residual: {residuals.max():.2f}")

# Percentage of predictions within certain error ranges
error_percentages = {}
for pct in [5, 10, 15, 20]:
    threshold = y_test.mean() * (pct / 100)
    within_threshold = (np.abs(residuals) <= threshold).sum()
    error_percentages[pct] = (within_threshold / len(y_test)) * 100

print(f"\nPrediction Accuracy:")
for pct, accuracy in error_percentages.items():
    print(f"  * Within {pct}% of actual: {accuracy:.1f}% of predictions")

## 8. Save the Final Model

In [ ]:
print(f"\nSAVING THE FINAL MODEL")
print("=" * 40)

# Create models directory if it doesn't exist
import os
os.makedirs('../models', exist_ok=True)

# Save the best model
model_path = f'../models/best_model_{best_model_name.lower().replace(" ", "_")}.pkl'
joblib.dump(best_model, model_path)
print(f"Best model saved: {model_path}")

# Save model metadata
model_metadata = {
    'model_name': best_model_name,
    'model_type': str(type(best_model).__name__),
    'test_r2_score': float(best_r2),
    'test_rmse': float(results_df.loc[best_model_name, 'Test_RMSE']),
    'test_mae': float(results_df.loc[best_model_name, 'Test_MAE']),
    'feature_count': int(X.shape[1]),
    'training_samples': int(len(X_train)),
    'test_samples': int(len(X_test)),
    'target_column': target_col,
    'features_used': X.columns.tolist()
}

# Save metadata
with open('../models/model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2)

print(f"Model metadata saved: '../models/model_metadata.json'")

# Save performance results
results_df.to_csv('../models/model_comparison_results.csv')
print(f"Comparison results saved: '../models/model_comparison_results.csv'")

## Model Development Summary
Multiple machine learning models were trained, tuned, and compared using robust evaluation metrics. The best-performing model was selected based on validation results and error analysis. This model, along with insights into feature importance and error patterns, provides a strong foundation for final evaluation and deployment in the next stage.